# Dataset Introduction - the Project Knowledge Graph

Ruijie Wang, Pascal Severin Andermatt, Rosni Vasu | 25-09-2023  
Matthias Baumgartner, Luca Rossetto, Cristina Sarasua | Dataset Construction

## 1. Overview

* The knowledge graph is
    * mainly about **movies**,
    * derived from **[Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)**,
    * **directed** and **connected**.


* Entities (movies, people, etc.)
    * common properties 
        * **labels** (rdfs:label)
        * **types** (wd:P31)
        * **descriptions** (schema:description)
    * namespace:
        * **wikidata_entity**(http://www.wikidata.org/entity/);
    
    
* Predicates
    * common properties
        * **labels** (rdfs:label)
        * **descriptions** (schema:description)
    * namespaces: 
        * **wikidata_property**(http://www.wikidata.org/prop/direct/)
        * **schema.org**(http://schema.org/)
        * **ddis.ch**(http://ddis.ch/atai/)
        * **rdfs**(http://www.w3.org/2000/01/rdf-schema#)
    
    
* Literal types in the graph (for some entities):
    * entity/predicate labels
    * descriptions
    * [IMDb](https://www.imdb.com/) ratings
    * box office values
    * publication dates
    * WikiCommons image URLs
    * user-defined tags
    
    
* Linked external datasets (for some entities):
    * movie plot descriptions from [MPST](https://www.kaggle.com/cryptexcode/mpst-movie-plot-synopses-with-tags) and [OMDb](http://www.omdbapi.com/),
    * textual user reviews from [LMRD](https://ai.stanford.edu/~amaas/data/sentiment/),
    * images of movies (e.g., posters) and people (e.g., still images) from [MovieNet](http://movienet.site/);
    
    
**The dataset is published at https://files.ifi.uzh.ch/ddis/teaching/2023/ATAI/dataset/**

## 2. Data Analysis


### 2.1 Load the data

In [1]:
!python3 -m pip install rdflib
!python3 -m pip install networkx
!python3 -m pip install pandas
!python3 -m pip install plotly
!python3 -m pip install -U kaleido

In [1]:
from rdflib.namespace import Namespace, RDF, RDFS, XSD
from rdflib.term import URIRef, Literal
import csv
import json
import networkx as nx
import pandas as pd
import rdflib
from collections import defaultdict, Counter
import locale
_ = locale.setlocale(locale.LC_ALL, '')
from _plotly_future_ import v4_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'jupyterlab+svg'

In [2]:
graph = rdflib.Graph()
graph.parse('/Users/melihserin/Desktop/ATAI/dataset/14_graph.nt', format='turtle') # change the path

<Graph identifier=N4fb4644a03684f9ea2c2b1ac16bf898f (<class 'rdflib.graph.Graph'>)>

### 2.2 Graph Statistics

In [3]:
# prefixes used in the graph
WD = Namespace('http://www.wikidata.org/entity/')
WDT = Namespace('http://www.wikidata.org/prop/direct/')
SCHEMA = Namespace('http://schema.org/')
DDIS = Namespace('http://ddis.ch/atai/')

In [4]:
entities = set(graph.subjects()) | {s for s in graph.objects() if isinstance(s, URIRef)}
predicates = set(graph.predicates())
literals = {s for s in graph.objects() if isinstance(s, Literal)}
with_type = set(graph.subjects(WDT['P31'], None))
with_super = set(graph.subjects(WDT['P279'], None))
types = set(graph.objects(None, WDT['P31']))
supers = set(graph.objects(None, WDT['P279']))
with_label = set(graph.subjects(RDFS.label, None))

n_ents = len(entities)
n_rels = len(predicates)
n_lits = len(literals)
t_tot = len(graph)
t_ent = len([1 for s,p,o in graph.triples((None, None, None)) if isinstance(o, URIRef)])
t_lit = t_tot - t_ent
n_notype = len(entities - with_type - with_super)
n_notype_flt = len(entities - with_type - with_super - types - supers)

pd.DataFrame([
    ('number of entities', f'{n_ents:n}'),
    ('number of literals', f'{n_lits:n}'),
    ('number of predicates', f'{n_rels:n}'),
    ('number of triples', f'{t_tot:n}'),
    ('number of ent-ent triples', f'{t_ent:n}'),
    ('number of ent-lit triples', f'{t_lit:n}'),
    ('number of entities w/o label', f'{len(entities - with_label):n}'),
    ('number of predicates w/o label', f'{len(predicates - with_label):n}'),
    ('number of entities w/o type', f'{n_notype:n}'),
    ('number of instances w/o type', f'{n_notype_flt:n}'),
    ])

,0,1
0,number of entities,159154
1,number of literals,411590
2,number of predicates,255
3,number of triples,2056777
4,number of ent-ent triples,1498899
5,number of ent-lit triples,557878
6,number of entities w/o label,1095
7,number of predicates w/o label,2
8,number of entities w/o type,5761
9,number of instances w/o type,344


In [5]:
roots = {
    WD['Q8242']:        'literature',
    WD['Q5']:           'human',
    WD['Q483394']:      'genre',
    WD['Q95074']:       'character',
    WD['Q11424']:       'film',
    WD['Q15416']:       'tv',
    WD['Q618779']:      'award',
    WD['Q27096213']:    'geographic',
    WD['Q43229']:       'organisation',
    WD['Q34770']:       'language',
    WD['Q7725310']:     'series',
    WD['Q47461344']:    'written work',
}

## 3. SPARQL query examples

In [6]:
# top user-rated movies
[str(s) for s, in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        SELECT ?movie ?lbl ?rating WHERE {
            ?movie wdt:P31 wd:Q11424 .
            ?movie ddis:rating ?rating .
            ?movie rdfs:label ?lbl .
        }
        ORDER BY DESC(?rating) 
        LIMIT 20
    }
    ''')]

['Forrest Gump',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Heart of a Dog',
 'Kannathil Muthamittal',
 'Once Upon a Time in America',
 'Oldboy',
 'The Great Dictator',
 'Apocalypse Now',
 'The Shining',
 'The Cranes Are Flying',
 'Shwaas',
 'Khosla Ka Ghosla',
 'Inglourious Basterds',
 'Good Will Hunting',
 'Full Metal Jacket',
 'The Ascent',
 'War and Peace',
 '2001: A Space Odyssey',
 'Scarface',
 'Star Wars: Episode VI – Return of the Jedi']

In [7]:
# bottom user-rated movies
[str(s) for s, in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        SELECT ?movie ?lbl ?rating WHERE {
            ?movie wdt:P31 wd:Q11424 .
            ?movie ddis:rating ?rating .
            ?movie rdfs:label ?lbl .
        }
        ORDER BY ASC(?rating) 
        LIMIT 10
    }
    ''')]

['Vampire Assassin',
 'Vampires vs. Zombies',
 'Aag',
 'Joystick Nation – Generation Hip Hop',
 'Going Overboard',
 "Alex l'ariete",
 'House of the Dead',
 'Killers',
 "Ghosts Can't Do It",
 'Snakes on a Train']

In [8]:
# some info about a Apocalypse Now

header = '''
    PREFIX ddis: <http://ddis.ch/atai/>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX schema: <http://schema.org/>
'''

tuple_list = list(graph.query(header + '''
    SELECT * WHERE {
        ?movie rdfs:label "Apocalypse Now"@en .
        ?movie wdt:P57/rdfs:label ?director .
        OPTIONAL { ?movie ddis:rating ?rating } .
        OPTIONAL { ?movie wdt:P577 ?value}
    }
    '''))

first_tuple = tuple_list[0]

print(f"First tuple: {first_tuple}")
print('------------')

for elements in first_tuple:
    print(elements)

First tuple: (rdflib.term.URIRef('http://www.wikidata.org/entity/Q182692'), rdflib.term.Literal('Francis Ford Coppola', lang='en'), rdflib.term.Literal('8.4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')), rdflib.term.Literal('1979-05-10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
------------
http://www.wikidata.org/entity/Q182692
Francis Ford Coppola
8.4
1979-05-10


In [19]:
# dealing with optional parameters
tuple_list = list(graph.query(header + '''
    SELECT ?lbl ?rating WHERE {
        ?movie rdfs:label ?lbl .
        ?movie wdt:P57/rdfs:label ?director .
        OPTIONAL { ?movie ddis:rating ?rating } .
        OPTIONAL { ?movie wdt:P577 ?value}
    }
    LIMIT 10
    '''))

# unpacking the tuple in the loop
for (movie_label, rating) in tuple_list:
    if rating:
        print(f"{movie_label} has a rating of {rating} ⭐️")
    else:
        print(f"{movie_label} has no rating 😔")

Jan Dara has no rating 😔
Queens of Langkasuka has no rating 😔
Three has no rating 😔
Moondram Pirai has no rating 😔
Buffalo Bill and the Indians, or Sitting Bull's History Lesson has no rating 😔
Dr. T & the Women has no rating 😔
McCabe & Mrs. Miller has a rating of 7.7 ⭐️
Nashville has no rating 😔
Fool for Love has a rating of 6.1 ⭐️
The Gingerbread Man has a rating of 5.7 ⭐️


In [20]:
# all movies directed by Terry Gilliam
[str(s) for s,  in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        ?director rdfs:label "Terry Gilliam"@en .
        ?movie wdt:P57 ?director .
        ?movie rdfs:label ?lbl
    }
    ''')]

['Time Bandits',
 'Tideland',
 "Monty Python's The Meaning of Life",
 'Brazil',
 'The Wholly Family',
 'The Fisher King',
 'The Imaginarium of Doctor Parnassus',
 'The Zero Theorem',
 'The Adventures of Baron Munchausen',
 'Jabberwocky',
 '12 Monkeys',
 'The Man Who Killed Don Quixote',
 'The Brothers Grimm',
 'Fear and Loathing in Las Vegas',
 'Monty Python and the Holy Grail']

In [21]:
# neo-noir movies featuring Ryan Gosling
[str(s) for s, in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        ?genre rdfs:label "neo-noir"@en .
        ?actor rdfs:label "Ryan Gosling"@en .
        ?movie wdt:P136 ?genre .
        ?movie wdt:P161 ?actor .
        ?movie rdfs:label ?lbl .
    }
    ''')]

['Only God Forgives', 'Drive', 'Blade Runner 2049']

In [22]:
# movies with largest cast member list
[(str(s), int(nc)) for s, nc in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl ?nc WHERE {
        SELECT ?movie ?lbl (count(?cast) as ?nc) WHERE {
            ?movie wdt:P31 wd:Q11424 .
            ?movie rdfs:label ?lbl .
            ?movie wdt:P161 ?cast .
        }
        GROUP BY ?movie
        ORDER BY DESC(?nc)
        LIMIT 10
    }
    ''')]

[('Mamma Mia! Here We Go Again', 224),
 ('Ali', 121),
 ('Forrest Gump', 118),
 ('Terror in the Aisles', 110),
 ('Iron Man 3', 108),
 ('The Longest Day', 104),
 ('Avengers: Endgame', 100),
 ('Captain America: Civil War', 98),
 ('Around the World in 80 Days', 93),
 ('Captain America: The First Avenger', 93)]

In [23]:
# cast of Moon
[str(s) for s, in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        ?movie rdfs:label "Moon"@en .
        ?movie wdt:P161 ?cast .
        ?cast rdfs:label ?lbl .
    }
    ''')]

['Matt Berry',
 'Kaya Scodelario',
 'Kevin Spacey',
 'Sam Rockwell',
 'Benedict Wong',
 'Dominique McElligott',
 'Robin Chalk']

In [24]:
# winners of Cannes best movie (Palme d'Or)
a = [(str(d), str(s)) for s, d in graph.query(header + '''
    SELECT ?lbl ?pubdate WHERE {
        ?award rdfs:label "Palme d'Or"@en .
        ?movie wdt:P166 ?award .
        ?movie rdfs:label ?lbl .
        ?movie wdt:P577 ?pubdate .
        FILTER (?pubdate > "2011-01-01"^^xsd:date)
    }
    ORDER BY DESC(?pubdate)
    ''')]

# this can be also written as (notice the ";"):
b = [(str(d), str(s)) for s, d in graph.query(header + '''
    SELECT ?lbl ?pubdate WHERE {
      ?award rdfs:label "Palme d'Or"@en.
      ?movie wdt:P166 ?award; rdfs:label ?lbl; wdt:P577 ?pubdate.
      FILTER(?pubdate > "2011-01-01"^^xsd:date)
    }
    ORDER BY DESC (?pubdate)
    ''')]

assert (a == b)
a

[('2021-07-14', 'Titane'),
 ('2019-05-21', 'Parasite'),
 ('2018-05-13', 'Shoplifters'),
 ('2017-05-20', 'The Square'),
 ('2016-10-21', 'I, Daniel Blake'),
 ('2015-10-22', 'Dheepan'),
 ('2014-05-16', 'Winter Sleep'),
 ('2013-05-23', 'Blue Is the Warmest Colour'),
 ('2012-01-01', 'Amour'),
 ('2011-05-16', 'The Tree of Life')]

## 4.Agent Demo

### Evaluation 1

In [9]:
qres=graph.query("""
PREFIX ddis: <http://ddis.ch/atai/>   
PREFIX wd: <http://www.wikidata.org/entity/>   
PREFIX wdt: <http://www.wikidata.org/prop/direct/>   
PREFIX schema: <http://schema.org/>   

SELECT ?lbl WHERE {  
    ?movie wdt:P31 wd:Q11424 .  
    ?movie ddis:rating ?rating .  
    ?movie rdfs:label ?lbl .  
}  
ORDER BY DESC(?rating)   
LIMIT 1 
    """
)

for row in qres:
    answer = row.lbl
    print([str(answer)])

['Forrest Gump']


In [10]:
# Which movie has the lowest user rating? 

qres=graph.query(
''' 

PREFIX ddis: <http://ddis.ch/atai/>   

PREFIX wd: <http://www.wikidata.org/entity/>   

PREFIX wdt: <http://www.wikidata.org/prop/direct/>   

PREFIX schema: <http://schema.org/>   

  

SELECT ?lbl WHERE {  

    ?movie wdt:P31 wd:Q11424 .  

    ?movie ddis:rating ?rating .  

    ?movie rdfs:label ?lbl .  

}  

ORDER BY ASC(?rating)   

LIMIT 1 

    ''' )

for row in qres:
    answer = row.lbl
    print([str(answer)])

['Vampire Assassin']


In [11]:
# Who directed the movie Apocalypse Now? 

qres=graph.query("""

 PREFIX ddis: <http://ddis.ch/atai/>   

PREFIX wd: <http://www.wikidata.org/entity/>   

PREFIX wdt: <http://www.wikidata.org/prop/direct/>   

PREFIX schema: <http://schema.org/>   

  

SELECT ?director ?movie WHERE {  

    ?movie rdfs:label "Apocalypse Now"@en .  

        ?movie wdt:P57 ?directorItem . 

    ?directorItem rdfs:label ?director . 

}  

LIMIT 1  
"""  )

for row in qres:
    answer = row.director
    print([str(answer)])

['Francis Ford Coppola']


In [57]:
from speakeasypy import Speakeasy, Chatroom
from typing import List
import time

DEFAULT_HOST_URL = 'https://speakeasy.ifi.uzh.ch'
listen_freq = 2


class Agent:
    def __init__(self, username, password):
        self.username = username
        # Initialize the Speakeasy Python framework and login.
        self.speakeasy = Speakeasy(host=DEFAULT_HOST_URL, username=username, password=password)
        self.speakeasy.login()  # This framework will help you log out automatically when the program terminates.

    def listen(self):
        while True:
            # only check active chatrooms (i.e., remaining_time > 0) if active=True.
            rooms: List[Chatroom] = self.speakeasy.get_rooms(active=True)
            for room in rooms:
                if not room.initiated:
                    # send a welcome message if room is not initiated
                    room.post_messages(f'Hello! This is a welcome message from {room.my_alias}.')
                    room.initiated = True
                # Retrieve messages from this chat room.
                # If only_partner=True, it filters out messages sent by the current bot.
                # If only_new=True, it filters out messages that have already been marked as processed.
                for message in room.get_messages(only_partner=True, only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new message #{message.ordinal}: '{message.message}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #
                    if message[message.find("SELECT")+8:message.find("WHERE")-1].find(" ") == -1:
                        row_names = [message[message.find("SELECT")+8:message.find("WHERE")-1]]
                    else:
                        row_names = message[message.find("SELECT")+8:message.find("WHERE")-1].replace("?","").split(" ")
                    qres=graph.query(message)
                    for row in qres:
                        answer=[]
                        for row_name in row_names:
                            answer.append(row[row_name])
                    # Send a message to the corresponding chat room using the post_messages method of the room object.
                    room.post_messages(f"Received your message: '{answer}' ")
                    # Mark the message as processed, so it will be filtered out when retrieving new messages.
                    room.mark_as_processed(message)

                # Retrieve reactions from this chat room.
                # If only_new=True, it filters out reactions that have already been marked as processed.
                for reaction in room.get_reactions(only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new reaction #{reaction.message_ordinal}: '{reaction.type}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #

                    room.post_messages(f"Received your reaction: '{reaction.type}' ")
                    room.mark_as_processed(reaction)

            time.sleep(listen_freq)

    @staticmethod
    def get_time():
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())


if __name__ == '__main__':
    demo_bot = Agent("bot_name", "bot_pass")
    demo_bot.listen()
